In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [765]:
df_train_set = pd.read_csv('train_aggregated.csv')

In [766]:
df_train_set.head()

,ride_id,travel_date,travel_time,travel_from,car_type,max_capacity,number_of_tickets
0,1442,17-10-17,7:15,Migori,Bus,49,1.0
1,5437,19-11-17,7:12,Migori,Bus,49,1.0
2,5710,26-11-17,7:05,Keroka,Bus,49,1.0
3,5777,27-11-17,7:10,Homa Bay,Bus,49,5.0
4,5778,27-11-17,7:12,Migori,Bus,49,31.0


In [767]:
df_train_set.drop(["ride_id"], axis=1, inplace=True)

In [768]:
df_train_set["travel_date"] = pd.to_datetime(df_train_set["travel_date"],infer_datetime_format=True)
df_train_set["travel_day"] = df_train_set["travel_date"].dt.dayofyear

In [769]:
df_train_set["car_type"] = pd.Categorical(df_train_set["car_type"])
car_type_categories = df_train_set.car_type.cat.categories
df_train_set["car_type"] = df_train_set.car_type.cat.codes

In [770]:
df_train_set["travel_from"] = pd.Categorical(df_train_set["travel_from"])
travel_from_categories = df_train_set.travel_from.cat.categories
df_train_set["travel_from"] = df_train_set.travel_from.cat.codes

In [771]:
#express travel time in hours
df_train_set["travel_time"] = df_train_set["travel_time"].str.split(':').apply(lambda x: int(x[0]) + (int(x[1])/60))

In [703]:
#num_of_tickets per hour
ticks = df_train_set.groupby("travel_time")["number_of_tickets"].count()
df_train_set["number_of_tickets"] = np.log1p(df_train_set["travel_time"].map(ticks))''''''

In [772]:
df_train_set.head()

,travel_date,travel_time,travel_from,car_type,max_capacity,number_of_tickets,travel_day
0,2017-10-17,7.250000,9,0,49,1.0,290
1,2017-11-19,7.200000,9,0,49,1.0,323
2,2017-11-26,7.083333,4,0,49,1.0,330
3,2017-11-27,7.166667,1,0,49,5.0,331
4,2017-11-27,7.200000,9,0,49,31.0,331


In [773]:
df_train_set['number_of_tickets'] = np.log1p(df_train_set['number_of_tickets'])

In [774]:
df_train_set = df_train_set.set_index('travel_date')
df_train_set.head()

,travel_time,travel_from,car_type,max_capacity,number_of_tickets,travel_day
travel_date,,,,,,
2017-10-17,7.250000,9,0,49,0.693147,290
2017-11-19,7.200000,9,0,49,0.693147,323
2017-11-26,7.083333,4,0,49,0.693147,330
2017-11-27,7.166667,1,0,49,1.791759,331
2017-11-27,7.200000,9,0,49,3.465736,331


In [775]:
df_train_set.groupby(['travel_from', 'travel_time', 'travel_day'], sort=False)

In [776]:
df_train_set['period'] = np.nan

In [777]:
df_train_set.loc[df_train_set.travel_time < 7, 'period'] = 'em'
df_train_set.loc[(df_train_set.travel_time >= 7) & (df_train_set.travel_time < 11), 'period'] = 'am'
df_train_set.loc[(df_train_set.travel_time >= 11) & (df_train_set.travel_time < 15), 'period'] = 'mid'
df_train_set.loc[(df_train_set.travel_time >= 15) & (df_train_set.travel_time < 19), 'period'] = 'eve'
df_train_set.loc[(df_train_set.travel_time >= 19) & (df_train_set.travel_time <= 24), 'period'] = 'pm'

In [778]:
df_train_set["period"] = pd.Categorical(df_train_set["period"])
period_categories = df_train_set.period.cat.categories
df_train_set["period"] = df_train_set.period.cat.codes

In [779]:
tcount = dict(df_train_set["period"].value_counts())
tcount

{0: 4888, 1: 988, 3: 286, 2: 87}

In [780]:
df_train_set["hourly_travelers"] = np.log1p(df_train_set["period"].map(tcount))
df_train_set.head()

,travel_time,travel_from,car_type,max_capacity,number_of_tickets,travel_day,period,hourly_travelers
travel_date,,,,,,,,
2017-10-17,7.250000,9,0,49,0.693147,290,0,8.494743
2017-11-19,7.200000,9,0,49,0.693147,323,0,8.494743
2017-11-26,7.083333,4,0,49,0.693147,330,0,8.494743
2017-11-27,7.166667,1,0,49,1.791759,331,0,8.494743
2017-11-27,7.200000,9,0,49,3.465736,331,0,8.494743


In [808]:
tcount = dict(df_train_set["travel_day"].value_counts())
df_train_set["daily_travelers"] = np.log1p(df_train_set["travel_day"].map(tcount))
df_train_set.head()

,travel_time,travel_from,car_type,max_capacity,number_of_tickets,travel_day,period,hourly_travelers,daily_travelers
travel_date,,,,,,,,,
2017-10-17,7.250000,9,0,49,0.693147,290,0,8.494743,0.693147
2017-11-19,7.200000,9,0,49,0.693147,323,0,8.494743,0.693147
2017-11-26,7.083333,4,0,49,0.693147,330,0,8.494743,0.693147
2017-11-27,7.166667,1,0,49,1.791759,331,0,8.494743,3.871201
2017-11-27,7.200000,9,0,49,3.465736,331,0,8.494743,3.871201


In [809]:
df_train_set.shape

(6249, 9)

In [810]:
""" Separate the data into features and targets """
target_var = ['number_of_tickets']
features, targets = df_train_set.drop(target_var, axis=1), df_train_set[target_var]

In [811]:
train_X, val_X, train_y, val_y = train_test_split(features, targets, test_size=0.2, random_state=0)

# Random forest model

In [812]:
X = train_X
y = train_y

In [718]:
#identifying optimum features
def get_mae(max_leaf_nodes, X, val_X, y, val_y):
    fmod = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=42, criterion='mae', n_estimators=1000, n_jobs=-1)
    fmod.fit(X, y)
    preds = fmod.predict(val_X)
    ma_e = mean_absolute_error(np.expm1(val_y), np.expm1(preds))
    return ma_e

In [ ]:
for max_leaf_nodes in [10,100,120,140,200,400]:
    my_mae = get_mae(max_leaf_nodes, X, val_X, y, val_y)
    print(f"Max_leaf_nodes: {max_leaf_nodes}, \t MAE: {my_mae}")

In [813]:
model = RandomForestRegressor(criterion='mae', max_depth=100, max_leaf_nodes=900, 
                              min_samples_leaf= 3, min_samples_split=8,
                              n_estimators=1500, n_jobs=-1, random_state=42)

In [814]:
model.fit(X,y)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=100,
           max_features='auto', max_leaf_nodes=900,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=8,
           min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [815]:
preds_train_set = model.predict(val_X)

In [816]:
print (mean_absolute_error(np.expm1(preds_train_set), np.expm1(val_y)))

3.273359441266504


# Predictions for test set

In [817]:
df_test_set = pd.read_csv('test_questions.csv')

In [818]:
df_test_set.drop(['travel_to'], axis=1, inplace=True)

In [819]:
df_test_set["travel_date"] = pd.to_datetime(df_test_set["travel_date"],infer_datetime_format=True)
df_test_set["travel_day"] = df_test_set["travel_date"].dt.dayofyear

In [820]:
df_test_set["car_type"] = pd.Categorical(df_test_set["car_type"], categories=car_type_categories)
df_test_set["car_type"] = df_test_set.car_type.cat.codes

In [821]:
df_test_set["travel_from"] = pd.Categorical(df_test_set["travel_from"], categories=travel_from_categories)
df_test_set["travel_from"] = df_test_set.travel_from.cat.codes

In [822]:
df_test_set["travel_time"] = df_test_set["travel_time"].str.split(':').apply(lambda x: int(x[0]) + int(x[1])/60)

In [823]:
df_test_set = df_test_set.set_index('travel_date')
df_test_set.head()

,ride_id,travel_time,travel_from,car_type,max_capacity,travel_day
travel_date,,,,,,
2018-05-07,247,7.100000,7,0,49,127
2018-05-06,256,11.133333,7,1,11,126
2018-05-04,275,5.000000,7,1,11,124
2018-05-04,285,9.166667,7,1,11,124
2018-05-04,286,9.333333,7,1,11,124


In [824]:
df_test_set.groupby(['travel_time', 'travel_from', 'travel_day'])

In [825]:
df_test_set['period'] = np.nan

In [826]:
df_test_set.loc[df_test_set.travel_time < 7, 'period'] = 'em'
df_test_set.loc[(df_test_set.travel_time >= 7) & (df_test_set.travel_time < 11), 'period'] = 'am'
df_test_set.loc[(df_test_set.travel_time >= 11) & (df_test_set.travel_time < 15), 'period'] = 'mid'
df_test_set.loc[(df_test_set.travel_time >= 15) & (df_test_set.travel_time < 19), 'period'] = 'eve'
df_test_set.loc[(df_test_set.travel_time >= 19) & (df_test_set.travel_time <= 24), 'period'] = 'pm'

In [827]:
pcount = dict(df_test_set['period'].value_counts())
pcount

{'am': 791, 'em': 145, 'pm': 123, 'mid': 52}

In [828]:
df_test_set['hourly_travelers'] = np.log1p(df_test_set['period'].map(pcount))

In [829]:
dcount = dict(df_test_set["travel_day"].value_counts())
df_test_set["daily_travelers"] = np.log1p(df_test_set["travel_day"].map(dcount))
df_test_set.head()

,ride_id,travel_time,travel_from,car_type,max_capacity,travel_day,period,hourly_travelers,daily_travelers
travel_date,,,,,,,,,
2018-05-07,247,7.100000,7,0,49,127,am,6.674561,1.945910
2018-05-06,256,11.133333,7,1,11,126,mid,3.970292,2.564949
2018-05-04,275,5.000000,7,1,11,124,em,4.983607,4.094345
2018-05-04,285,9.166667,7,1,11,124,am,6.674561,4.094345
2018-05-04,286,9.333333,7,1,11,124,am,6.674561,4.094345


In [830]:
df_test_set["period"] = pd.Categorical(df_test_set["period"])
period_categories = df_test_set.period.cat.categories
df_test_set["period"] = df_test_set.period.cat.codes

Now let's calculate predictions using the random forest model we trained.

In [831]:
X_test = df_test_set.drop(['ride_id'], axis=1)
test_set_predictions = model.predict(X_test)

And finally let's create a csv file with predictions. 

In [832]:
d = {'ride_id': df_test_set.ride_id, 'number_of_ticket': np.round(np.expm1(test_set_predictions), 0)}
df_predictions = pd.DataFrame(data=d)

In [833]:
df_predictions.head()

,ride_id,number_of_ticket
travel_date,,
2018-05-07,247,3.0
2018-05-06,256,1.0
2018-05-04,275,1.0
2018-05-04,285,9.0
2018-05-04,286,9.0


In [834]:
df_predictions.to_csv('zindi_submission_file.csv', index=False) #save to csv file